In [ ]:
%pip install llama-index-graph-stores-neo4j
%pip install neo4j
%pip install llama-index-packs-neo4j-query-engine
%pip install llama-index-tools-neo4j

In [ ]:
import sys
sys.path.append("../")
from IPython.display import Markdown, display
from llama_index.core import Settings
from common.env import AppConfig
from common.env import LocalLLM
from llama_index.core import Settings
config = AppConfig(local_llm=LocalLLM.LM_STUDIO)
logger = config.logger



In [ ]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

username = "neo4j"
password = "neo4j@sh123"
url = "bolt://localhost:7687"
database = "movies"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)
retriver = KnowledgeGraphRAGRetriever(storage_context=storage_context)
# Clean dataset first
# graph_store.query(
#     """
# MATCH (n) DETACH DELETE n
# """
# )

In [ ]:
from llama_index.core.prompts.base import BasePromptTemplate
# custom_prompt_template = """
# Your custom prompt template goes here.
# """
# custom_prompt = BasePromptTemplate(custom_prompt_template)
q = "tell me about Jack Chain"

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
engine = RetrieverQueryEngine(retriever=retriver)
resp = engine.get_prompts()
display(Markdown(f"<b>{resp}</b>"))

In [ ]:
from llama_index.tools.neo4j import Neo4jQueryToolSpec
from llama_index.agent.openai import OpenAIAgent

gds_db = Neo4jQueryToolSpec(
    url=url,
    user=username,
    password=password,
    llm=Settings.llm,
    database=database,
)

tools = gds_db.to_tool_list()
agent = OpenAIAgent.from_tools(tools, verbose=True)
# q = "when was Gene Hackman born."
# q = "list all Ethan Hawke's movies."
# q = "how many actors are there in the movie When Harry Met Sally "
# q = "tell me about the movie When Harry Met Sally"
q = "list James Thompson's television series"
resp = agent.query(q)
display(Markdown(f"<b>{resp}</b>"))

In [ ]:

from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.core import PropertyGraphIndex

engine = KnowledgeGraphQueryEngine(storage_context=storage_context, 
                                   llm=Settings.llm, verbose=True)
resp = engine.query("how many movies has Tom Hanks starred in")
display(Markdown(f"<b>{resp}</b>"))

In [ ]:
from llama_index.packs.neo4j_query_engine import Neo4jQueryEnginePack
from llama_index.packs.neo4j_query_engine.base import Neo4jQueryEngineType

neo4j_pack = Neo4jQueryEnginePack(
    username=username,
    password=password,
    url=url,
    database=database,
    query_engine_type=Neo4jQueryEngineType.KG_HYBRID,
)